In [615]:
import pandas as pd
import numpy as np
import re
import math

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Import natural language processing libraries
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Model selection libraries
from sklearn.model_selection import train_test_split, GridSearchCV

# Model Evaluation Libraries
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Import pipeline library
from sklearn.pipeline import Pipeline

from tempfile import mkdtemp

In [381]:
# Import warnings and supress them
import warnings
warnings.filterwarnings('ignore')

In [382]:
# Configure Pandas to show all columns / rows
pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000
# Set column max width larger
pd.set_option('display.max_colwidth', 200)

In [585]:
# Set up matplotlib to use nice styles
plt.style.use('ggplot')